In [1]:
# # delete non-empty directories
# shutil.rmtree('results/hash')

In [2]:
# # Load librarires
import os
from pathlib import Path
# import pandas as pd

# Assign directories
current_dir = Path(os.getcwd()).absolute()
assignment_dir = current_dir.parent
assign03_dir = assignment_dir.joinpath('assignment03')
results_dir = assign03_dir.joinpath('results')

# parquet_path = results_dir.joinpath('routes.parquet')
# parquet_path
# df = pd.read_parquet('routes.parquet')
# pd.read_parquet(parquet_path)

In [3]:
current_dir

PosixPath('/home/jovyan/dsc650/dsc650/assignments/assignment07')

In [4]:
assignment_dir

PosixPath('/home/jovyan/dsc650/dsc650/assignments')

In [5]:
Path('/home/joyvan/dsc650/data/processed/openflights/routes.jsonl.gz')

PosixPath('/home/joyvan/dsc650/data/processed/openflights/routes.jsonl.gz')

## Assignment 7.1
In this part of the assignment, you will partition a dataset using different strategies. You will use the `routes.parquet` dataset you created in a previous assignment. For this dataset, the key for each route will be the three-letter source airport code concatenated with the three-letter destination airport code and the two-letter airline. For instance, a route from Omaha Eppley Airfield (OMA) to Denver International Airport (DEN) on American Airlines (AA) has a key of `OMADENAA`.

### a. 
Start by loading the dataset from the previous assignment using Pandas's [`read_parquet()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_parquet.html) method. Next, add the concatenated key then using Panda's [`apply()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) method to create a new column called `key`. 

In [6]:
current_dir

PosixPath('/home/jovyan/dsc650/dsc650/assignments/assignment07')

In [7]:
import os
import json
from pathlib import Path
import gzip
import hashlib
import shutil
import pandas as pd
import pygeohash
import s3fs
import uuid
import math
endpoint_url='https://storage.budsc.midwest-datascience.com'
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
if results_dir.exists():
    shutil.rmtree(results_dir)
results_dir.mkdir(parents=True, exist_ok=True)
# def read_jsonl_data():
# #     s3 = s3fs.S3FileSystem(
# #         anon=True,
# #         client_kwargs={
# #             'endpoint_url': endpoint_url
# #         }
# #     )
# #     src_data_path = 'data/processed/openflights/routes.jsonl.gz'
# #     with s3.open(src_data_path, 'rb') as f_gz:
# #         with gzip.open(f_gz, 'rb') as f:
# #             records = [json.loads(line) for line in f.readlines()]
#     src_data_path = '/home/jovyan/dsc650/data/processed/openflights/routes.jsonl.gz'
# #     src_data_path = 'data/routes.jsonl.gz'
#     with gzip.open(src_data_path, 'rb') as f:
#         records = [json.loads(line) for line in f.readlines()]
        
#     return records
# def flatten_record(record):
#     flat_record = dict()
#     for key, value in record.items():
#         if key in ['airline', 'src_airport', 'dst_airport']:
#             if isinstance(value, dict):
#                 for child_key, child_value in value.items():
#                     flat_key = '{}_{}'.format(key, child_key)
#                     flat_record[flat_key] = child_value
#         else:
#             flat_record[key] = value
    
#     return flat_record
# def create_flattened_dataset():
#     records = read_jsonl_data()
#     parquet_path = results_dir.joinpath('routes-flattened.parquet')
#     return pd.DataFrame.from_records([flatten_record(record) for record in records])
# df = create_flattened_dataset()
# df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)

In [8]:
def read_jsonl_data():
    src_data_path = '/home/jovyan/dsc650/data/processed/openflights/routes.jsonl.gz'
    with gzip.open(src_data_path, 'rb') as f:
        records = [json.loads(line) for line in f.readlines()]       
    return records

In [9]:
def flatten_record(record):
    flat_record = dict()
    for key, value in record.items():
        if key in ['airline', 'src_airport', 'dst_airport']:
            if isinstance(value, dict):
                for child_key, child_value in value.items():
                    flat_key = '{}_{}'.format(key, child_key)
                    flat_record[flat_key] = child_value
        else:
            flat_record[key] = value
    
    return flat_record
def create_flattened_dataset():
    records = read_jsonl_data()
    parquet_path = results_dir.joinpath('routes-flattened.parquet')
    return pd.DataFrame.from_records([flatten_record(record) for record in records])

In [10]:
df = create_flattened_dataset()
df['key'] = df['src_airport_iata'].astype(str) + df['dst_airport_iata'].astype(str) + df['airline_iata'].astype(str)

In [11]:
results_dir

PosixPath('/home/jovyan/dsc650/dsc650/assignments/assignment07/results')

In [12]:
df.key

0        AERKZN2B
1        ASFKZN2B
2        ASFMRV2B
3        CEKKZN2B
4        CEKOVB2B
           ...   
67658    WYAADLZL
67659    DMEFRUZM
67660    FRUDMEZM
67661    FRUOSSZM
67662    OSSFRUZM
Name: key, Length: 67663, dtype: object

In [13]:
df.columns

Index(['airline_airline_id', 'airline_name', 'airline_alias', 'airline_iata',
       'airline_icao', 'airline_callsign', 'airline_country', 'airline_active',
       'src_airport_airport_id', 'src_airport_name', 'src_airport_city',
       'src_airport_country', 'src_airport_iata', 'src_airport_icao',
       'src_airport_latitude', 'src_airport_longitude', 'src_airport_altitude',
       'src_airport_timezone', 'src_airport_dst', 'src_airport_tz_id',
       'src_airport_type', 'src_airport_source', 'dst_airport_airport_id',
       'dst_airport_name', 'dst_airport_city', 'dst_airport_country',
       'dst_airport_iata', 'dst_airport_icao', 'dst_airport_latitude',
       'dst_airport_longitude', 'dst_airport_altitude', 'dst_airport_timezone',
       'dst_airport_dst', 'dst_airport_tz_id', 'dst_airport_type',
       'dst_airport_source', 'codeshare', 'equipment', 'key'],
      dtype='object')

For this part of the example, we will create 16 partitions so that we can compare it to the partitions we create from hashed keys in the next part of the assignment. The partitions are determined by the first letter of the composite key using the following partitions.

In [14]:
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

In this case `('A', 'A')` means the folder should contain all of the routes whose composite key starts with `A`. Similarly, `('E', 'F')` should contain routes whose composite key starts with `E` or `F`.

The `results/kv` directory should contain the following folders.

In [15]:
# kv
# ├── kv_key=A
# ├── kv_key=B
# ├── kv_key=C-D
# ├── kv_key=E-F
# ├── kv_key=G-H
# ├── kv_key=I-J
# ├── kv_key=K-L
# ├── kv_key=M
# ├── kv_key=N
# ├── kv_key=O-P
# ├── kv_key=Q-R
# ├── kv_key=S-T
# ├── kv_key=U
# ├── kv_key=V
# ├── kv_key=W-X
# └── kv_key=Y-Z

An easy way to create this directory structure is to create a new key called `kv_key` from the `key` column and use the [`to_parquet()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html) method with `partition_cols=['kv_key']` to save a partitioned dataset.

In [16]:
df['key'].str[0]

0        A
1        A
2        A
3        C
4        C
        ..
67658    W
67659    D
67660    F
67661    F
67662    O
Name: key, Length: 67663, dtype: object

In [17]:
testKey = df['key'][0]
testKey

'AERKZN2B'

In [18]:
testKey[0]

'A'

In [19]:
partitions

(('A', 'A'),
 ('B', 'B'),
 ('C', 'D'),
 ('E', 'F'),
 ('G', 'H'),
 ('I', 'J'),
 ('K', 'L'),
 ('M', 'M'),
 ('N', 'N'),
 ('O', 'P'),
 ('Q', 'R'),
 ('S', 'T'),
 ('U', 'U'),
 ('V', 'V'),
 ('W', 'X'),
 ('Y', 'Z'))

In [20]:
type(partitions)

tuple

In [21]:
partition_dict = {}
for i in partitions:
#     print(i)
#     print(i[0])
#     print(i[1])
    if i[0] == i[1]:
        partition_dict[i] = i[0]
    else:
        partition_dict[i] = i[0] + '-' + i[1] 
    

In [22]:
partition_dict

{('A', 'A'): 'A',
 ('B', 'B'): 'B',
 ('C', 'D'): 'C-D',
 ('E', 'F'): 'E-F',
 ('G', 'H'): 'G-H',
 ('I', 'J'): 'I-J',
 ('K', 'L'): 'K-L',
 ('M', 'M'): 'M',
 ('N', 'N'): 'N',
 ('O', 'P'): 'O-P',
 ('Q', 'R'): 'Q-R',
 ('S', 'T'): 'S-T',
 ('U', 'U'): 'U',
 ('V', 'V'): 'V',
 ('W', 'X'): 'W-X',
 ('Y', 'Z'): 'Y-Z'}

In [23]:
testKey

'AERKZN2B'

In [24]:
dataKey = 'XERKZN2B'

In [25]:
for key, val in partition_dict.items():
#     print(key)
#     print(key[0])
#     print(key[1])
#     print(val)
    if dataKey[0] == key[0] or dataKey[0] == key[1]:
        print(val)

W-X


In [26]:
# Generate kv_key from key
def kv_key_gen(data_key):
    for key, val in partition_dict.items():
        if data_key[0] == key[0] or data_key[0] == key[1]:
            return val
    return ''

In [27]:
# Add kv_key column to df
df['kv_key'] = df['key'].apply(kv_key_gen)

In [28]:
df[['key', 'kv_key']]

,key,kv_key
0,AERKZN2B,A
1,ASFKZN2B,A
2,ASFMRV2B,A
3,CEKKZN2B,C-D
4,CEKOVB2B,C-D
...,...,...
67658,WYAADLZL,W-X
67659,DMEFRUZM,C-D
67660,FRUDMEZM,E-F
67661,FRUOSSZM,E-F


In [29]:
results_dir.joinpath('kv')

PosixPath('/home/jovyan/dsc650/dsc650/assignments/assignment07/results/kv')

In [30]:
df.to_parquet(results_dir.joinpath('kv'), partition_cols=['kv_key'])

In [31]:
# Find rows with null keys
df[df['kv_key'] == '']

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_altitude,dst_airport_timezone,dst_airport_dst,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key
38,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,NaN,NaN,...,588.0,3.0,N,Europe/Moscow,airport,OurAirports,False,[CR2],nanDME2B,
54,1654,Cargoitalia,SN Brussels Airlines,2G,CRG,WHITE PELICAN,Italy,False,NaN,NaN,...,1675.0,8.0,N,Asia/Irkutsk,airport,OurAirports,False,[AN4],nanIKT2G,
175,146,Air Salone,\N,2O,nan,nan,Sierra Leone,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,[BNI],nannan2O,
176,146,Air Salone,\N,2O,nan,nan,Sierra Leone,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,[BNI],nannan2O,
178,146,Air Salone,\N,2O,nan,nan,Sierra Leone,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,False,[BNI],nannan2O,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67471,21,Aigle Azur,\N,ZI,AAF,AIGLE AZUR,France,True,NaN,NaN,...,3360.0,1.0,U,Africa/Algiers,airport,OurAirports,False,[320],nanQSFZI,
67537,2607,Great Lakes Airlines,FlyAsianXpress,ZK,GLA,LAKES AIR,United States,True,NaN,NaN,...,5431.0,-7.0,A,America/Denver,airport,OurAirports,False,[EM2],nanDENZK,
67542,2607,Great Lakes Airlines,FlyAsianXpress,ZK,GLA,LAKES AIR,United States,True,NaN,NaN,...,125.0,-8.0,A,America/Los_Angeles,airport,OurAirports,False,[BE1],nanLAXZK,
67543,2607,Great Lakes Airlines,FlyAsianXpress,ZK,GLA,LAKES AIR,United States,True,NaN,NaN,...,4316.0,-7.0,A,America/Phoenix,airport,OurAirports,False,[BE1],nanPGAZK,


### b.
Next, we are going to partition the dataset again, but this time we will partition by the hash value of the key. The following is a function that will create a SHA256 hash of the input key and return a hexadecimal string representation of the hash.

In [32]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [33]:
testKey

'AERKZN2B'

In [34]:
hash_key(testKey)

'652cdec02010381f175efe499e070c8cbaac1522bac59aa49ff86de6c8af6a30'

In [35]:
hash_key('42')

'73475cb40a568e8da8a045ced110137e159f890ac4da883b6b17dc651b3a8049'

We will partition the data using the first character of the hexadecimal hash. As such, there are 16 possible partitions. Create a new column called `hashed` that is a hashed value of the `key` column. Next, create a partitioned dataset based on the first character of the hashed key and save the results to `results/hash`. The directory should contain the following folders.

In [36]:
# hash
# ├── hash_key=0
# ├── hash_key=1
# ├── hash_key=2
# ├── hash_key=3
# ├── hash_key=4
# ├── hash_key=5
# ├── hash_key=6
# ├── hash_key=7
# ├── hash_key=8
# ├── hash_key=9
# ├── hash_key=A
# ├── hash_key=B
# ├── hash_key=C
# ├── hash_key=D
# ├── hash_key=E

In [37]:
# Add hash column to df
df['hashed'] = df['key'].apply(hash_key)

In [38]:
df['hashed']

0        652cdec02010381f175efe499e070c8cbaac1522bac59a...
1        9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...
2        161143856af25bd4475f62c80c19f68936a139f653c1d3...
3        39aa99e6ae2757341bede9584473906ef1089e30820c90...
4        143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...
                               ...                        
67658    f31527be84c36208c05cac57dfac8a46b48a87dda151f8...
67659    880fc35ca283ad034c90becc4e331b72ee894b9eb69f76...
67660    e976939986fbf947bb9318018cef717c0b34dff91e5e67...
67661    8b0c0b835a58a4250e020d51ec2a896e4ef3f5c3543b8e...
67662    629f14f3fb6f94ebd1522d33a3c50675942e3148d028b4...
Name: hashed, Length: 67663, dtype: object

In [39]:
df['hashed'][0][0]

'6'

In [40]:
testDF = pd.DataFrame()

In [41]:
testDF['hash_char'] = df['hashed'].str[0]

In [42]:
testDF

,hash_char
0,6
1,9
2,1
3,3
4,1
...,...
67658,f
67659,8
67660,e
67661,8


In [43]:
df['hk_hash'] = df['hashed'].str[0]

In [44]:
df[['hk_hash', 'hashed']]

,hk_hash,hashed
0,6,652cdec02010381f175efe499e070c8cbaac1522bac59a...
1,9,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...
2,1,161143856af25bd4475f62c80c19f68936a139f653c1d3...
3,3,39aa99e6ae2757341bede9584473906ef1089e30820c90...
4,1,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...
...,...,...
67658,f,f31527be84c36208c05cac57dfac8a46b48a87dda151f8...
67659,8,880fc35ca283ad034c90becc4e331b72ee894b9eb69f76...
67660,e,e976939986fbf947bb9318018cef717c0b34dff91e5e67...
67661,8,8b0c0b835a58a4250e020d51ec2a896e4ef3f5c3543b8e...


### c.
Finally, we will simulate multiple geographically distributed data centers. For this example, we will assume we have three data centers located in the western, central, and eastern United States. Google lists the [locations of their data centers](https://www.google.com/about/datacenters/locations/) and we will use the following locations for our three data centers.  
**West**
* The Dalles, Oregon
* Latitude: 45.5945645
* Longitude: -121.1786823  

**Central**
* Papillion, NE
* Latitude: 41.1544433
* Longitude: -96.0422378  

**East**
* Loudoun County, Virginia
* Latitude: 39.08344
* Longitude: -77.6497145  

Assume that you have an application that provides routes for each of the source airports and you want to store routes in the data center closest to the source airport. The output folders should look as follows.

In [45]:
# geo
# ├── location=central
# ├── location=east
# └── location=west

In [46]:
# Get geohash location info for data centers
west = pygeohash.encode(45.5945645, -121.1786823)
cent = pygeohash.encode(41.1544433, -96.0422378)
east = pygeohash.encode(39.08344, -77.6497145)

In [47]:
df.columns

Index(['airline_airline_id', 'airline_name', 'airline_alias', 'airline_iata',
       'airline_icao', 'airline_callsign', 'airline_country', 'airline_active',
       'src_airport_airport_id', 'src_airport_name', 'src_airport_city',
       'src_airport_country', 'src_airport_iata', 'src_airport_icao',
       'src_airport_latitude', 'src_airport_longitude', 'src_airport_altitude',
       'src_airport_timezone', 'src_airport_dst', 'src_airport_tz_id',
       'src_airport_type', 'src_airport_source', 'dst_airport_airport_id',
       'dst_airport_name', 'dst_airport_city', 'dst_airport_country',
       'dst_airport_iata', 'dst_airport_icao', 'dst_airport_latitude',
       'dst_airport_longitude', 'dst_airport_altitude', 'dst_airport_timezone',
       'dst_airport_dst', 'dst_airport_tz_id', 'dst_airport_type',
       'dst_airport_source', 'codeshare', 'equipment', 'key', 'kv_key',
       'hashed', 'hk_hash'],
      dtype='object')

In [48]:
df[['src_airport_latitude', 'src_airport_longitude']]

,src_airport_latitude,src_airport_longitude
0,43.449902,39.956600
1,46.283298,48.006302
2,46.283298,48.006302
3,55.305801,61.503300
4,55.305801,61.503300
...,...,...
67658,-33.058899,137.514008
67659,55.408798,37.906300
67660,43.061298,74.477600
67661,43.061298,74.477600


In [49]:
fn = lambda row: pygeohash.encode(row.src_airport_latitude, 
                                  row.src_airport_longitude)
testDF['geohash'] = df.apply(fn, axis=1)

In [50]:
testDF

,hash_char,geohash
0,6,szsrjjzd02b3
1,9,v04pk3t5gbjj
2,1,v04pk3t5gbjj
3,3,v3gdxs17du83
4,1,v3gdxs17du83
...,...,...
67658,f,r41gcjy9uwef
67659,8,ucfgnwfe8u9e
67660,e,txsuyz0fjzgd
67661,8,txsuyz0fjzgd


In [51]:
def get_geohash(df):
    geohash = pygeohash.encode(df['src_airport_latitude'], df['src_airport_longitude'])
    return geohash

In [52]:
df.apply(lambda row: pygeohash.encode(row.src_airport_latitude, 
                                      row.src_airport_longitude), axis=1)

0        szsrjjzd02b3
1        v04pk3t5gbjj
2        v04pk3t5gbjj
3        v3gdxs17du83
4        v3gdxs17du83
             ...     
67658    r41gcjy9uwef
67659    ucfgnwfe8u9e
67660    txsuyz0fjzgd
67661    txsuyz0fjzgd
67662    tx5z02wkwf2p
Length: 67663, dtype: object

In [53]:
# df.apply(lambda row: get_geohash(row), axis=1)

In [54]:
# testDF['geohash'] = df.apply(lambda row: get_geohash(row), axis=1)

In [55]:
# Get geohash location info for data centers
west = pygeohash.encode(45.5945645, -121.1786823)
cent = pygeohash.encode(41.1544433, -96.0422378)
east = pygeohash.encode(39.08344, -77.6497145)

In [56]:
datacenter_list = [west, cent, east]

In [57]:
# Get geohash location info for airports
fn = lambda row: pygeohash.encode(row.src_airport_latitude, 
                                  row.src_airport_longitude)
df['geohash'] = df.apply(fn, axis=1)

In [58]:
testHash = df['geohash'][67659]
testHash

'ucfgnwfe8u9e'

In [59]:
pygeohash.geohash_approximate_distance(testHash, west)/1000

20000.0

In [60]:
pygeohash.decode('ucfgnwfe8u9e')

(55.408798, 37.9063)

In [61]:
import math
from pygeohash.geohash import decode
from pygeohash.geohash import decode_exactly

In [62]:
pygeohash.geohash_approximate_distance('c21g6', 'dqby3')

20000000

In [63]:
pygeohash.geohash_haversine_distance('c21g6', 'dqby3')

3608927.096599867

In [64]:
for i in datacenter_list:
    print(i)
    print(pygeohash.geohash_haversine_distance(i, testHash))

c21g6s0rs4c7
8613682.033457367
9z7dnebnj8kb
8429962.737215051
dqby34cjw922
7872397.533938673


In [65]:
east

'dqby34cjw922'

In [66]:
# Get geohash location info for data centers
data_centers = {}
data_centers['west'] = pygeohash.encode(45.5945645, -121.1786823)
data_centers['central'] = pygeohash.encode(41.1544433, -96.0422378)
data_centers['east'] = pygeohash.encode(39.08344, -77.6497145)

In [67]:
data_centers

{'west': 'c21g6s0rs4c7', 'central': '9z7dnebnj8kb', 'east': 'dqby34cjw922'}

In [68]:
# testHash = pygeohash.encode(40.787560, -96.743690)

In [69]:
testHash = df['geohash'][1000]
testHash

'be1q5h5pjr4e'

In [70]:
for key, val in data_centers.items():
    print(key)
    print(pygeohash.geohash_haversine_distance(val, testHash))

west
2892060.42950635
central
4511694.3687240975
east
5622998.696937121


In [71]:
distance = 20000000

for key, val in data_centers.items():
    if pygeohash.geohash_haversine_distance(val, testHash) < distance:
        distance = pygeohash.geohash_haversine_distance(val, testHash)
        closest = key

In [72]:
closest

'west'

In [73]:
def find_closest_datacenter(airport_geohash):
    distance = 20000000
    closest = ''
    for key, val in data_centers.items():
        if pygeohash.geohash_haversine_distance(val, airport_geohash) < distance:
            closest = key
            distance = pygeohash.geohash_haversine_distance(val, airport_geohash)

    return closest

In [74]:
testDF = pd.DataFrame()

In [75]:
testDF['closest'] = df['geohash'].apply(find_closest_datacenter)

In [76]:
testDF['closest'].unique()

array(['east', 'west', 'central'], dtype=object)

In [77]:
df

,airline_airline_id,airline_name,airline_alias,airline_iata,airline_icao,airline_callsign,airline_country,airline_active,src_airport_airport_id,src_airport_name,...,dst_airport_tz_id,dst_airport_type,dst_airport_source,codeshare,equipment,key,kv_key,hashed,hk_hash,geohash
0,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2965.0,Sochi International Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],AERKZN2B,A,652cdec02010381f175efe499e070c8cbaac1522bac59a...,6,szsrjjzd02b3
1,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],ASFKZN2B,A,9eea5dd88177f8d835b2bb9cb27fb01268122b635b241a...,9,v04pk3t5gbjj
2,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2966.0,Astrakhan Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],ASFMRV2B,A,161143856af25bd4475f62c80c19f68936a139f653c1d3...,1,v04pk3t5gbjj
3,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,Europe/Moscow,airport,OurAirports,False,[CR2],CEKKZN2B,C-D,39aa99e6ae2757341bede9584473906ef1089e30820c90...,3,v3gdxs17du83
4,410,Aerocondor,ANA All Nippon Airways,2B,ARD,AEROCONDOR,Portugal,True,2968.0,Chelyabinsk Balandino Airport,...,Asia/Krasnoyarsk,airport,OurAirports,False,[CR2],CEKOVB2B,C-D,143b3389bce68eea3a13ac26a9c76c1fa583ec2bd26ea8...,1,v3gdxs17du83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,4178,Regional Express,Qantas Airways,ZL,RXA,REX,Australia,True,6334.0,Whyalla Airport,...,Australia/Adelaide,airport,OurAirports,False,[SF3],WYAADLZL,W-X,f31527be84c36208c05cac57dfac8a46b48a87dda151f8...,f,r41gcjy9uwef
67659,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,4029.0,Domodedovo International Airport,...,Asia/Bishkek,airport,OurAirports,False,[734],DMEFRUZM,C-D,880fc35ca283ad034c90becc4e331b72ee894b9eb69f76...,8,ucfgnwfe8u9e
67660,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,Europe/Moscow,airport,OurAirports,False,[734],FRUDMEZM,E-F,e976939986fbf947bb9318018cef717c0b34dff91e5e67...,e,txsuyz0fjzgd
67661,19016,Apache Air,Apache,ZM,IWA,APACHE,United States,True,2912.0,Manas International Airport,...,Asia/Bishkek,airport,OurAirports,False,[734],FRUOSSZM,E-F,8b0c0b835a58a4250e020d51ec2a896e4ef3f5c3543b8e...,8,txsuyz0fjzgd


In [78]:
df[['src_airport_latitude', 'src_airport_longitude']]

,src_airport_latitude,src_airport_longitude
0,43.449902,39.956600
1,46.283298,48.006302
2,46.283298,48.006302
3,55.305801,61.503300
4,55.305801,61.503300
...,...,...
67658,-33.058899,137.514008
67659,55.408798,37.906300
67660,43.061298,74.477600
67661,43.061298,74.477600


### d.
Create a Python function that takes as input a list of keys and the number of partitions and returns a list of keys sorted into the specified number of partitions. The partitions should be roughly equal in size. Furthermore, the partitions should have the property that each partition contains all the keys between the least key in the partition and the greatest key in the partition. In other words, the partitions should be ordered.

In [79]:
list_countries = list(df['airline_country'])

In [80]:
some_keys = ['S', 'C', 'O', 'T', 'T',
             'M', 'I', 'C', 'H', 'A', 'E', 'L', 
             'B', 'R', 'E', 'I', 'T', 'B', 'A', 'C', 'H']

In [81]:
# some_keys.sort()

In [82]:
# some_keys

In [83]:
def balance_partitions(keys, num_partitions):
    partitions = []
    keys.sort()
    num_keys = len(keys)
    partition_size = num_keys / num_partitions
    
    
    return partitions

In [84]:
def sharded(keys, num_partitions):
    for x in range(0, len(keys), num_partitions):
        chunk = keys[x:num_partitions+x]
        yield chunk

In [85]:
sharded(some_keys, 3)

<generator object sharded at 0x7fb1945da580>

In [86]:
test = sharded(some_keys, 3)

In [87]:
for i in test:
    print(i)

['S', 'C', 'O']
['T', 'T', 'M']
['I', 'C', 'H']
['A', 'E', 'L']
['B', 'R', 'E']
['I', 'T', 'B']
['A', 'C', 'H']


In [88]:
def sharded(keys, num_partitions):
    for x in range(0, len(keys), num_partitions):
        chunk = keys[x:num_partitions+x]
        yield(chunk)

In [89]:
sharded(some_keys, 3)

<generator object sharded at 0x7fb189291f20>

In [90]:
list(sharded(some_keys, 3))

[['S', 'C', 'O'],
 ['T', 'T', 'M'],
 ['I', 'C', 'H'],
 ['A', 'E', 'L'],
 ['B', 'R', 'E'],
 ['I', 'T', 'B'],
 ['A', 'C', 'H']]

In [91]:
# def balance_partitions(keys, num_partitions):
#     keys.sort()
#     num_keys = len(keys)
#     partition_size = num_keys / num_partitions
#     partitions = list(sharded(keys, num_per_group))
    
#     return partitions

In [92]:
def sharded(keys, num_partitions):
    for x in range(0, len(keys), num_partitions):
        chunk = keys[x:num_partitions+x]
        yield(chunk)

In [93]:
def balance_partitions(keys, num_partitions):
    keys.sort()
    num_per_group = math.ceil(len(keys) / num_partitions)
    return list(sharded(keys, num_per_group))

In [94]:
balance_partitions(some_keys, 3)

[['A', 'A', 'B', 'B', 'C', 'C', 'C'],
 ['E', 'E', 'H', 'H', 'I', 'I', 'L'],
 ['M', 'O', 'R', 'S', 'T', 'T', 'T']]

In [95]:
some_keys = ['S', 'C', 'O', 'T', 'T', 'Y',
             'M', 'I', 'C', 'H', 'A', 'E', 'L', 
             'B', 'R', 'E', 'I', 'T', 'B', 'A', 'C', 'H']

In [96]:
balance_partitions(some_keys, 3)

[['A', 'A', 'B', 'B', 'C', 'C', 'C', 'E'],
 ['E', 'H', 'H', 'I', 'I', 'L', 'M', 'O'],
 ['R', 'S', 'T', 'T', 'T', 'Y']]

In [97]:
def sharded(keys, num_partitions):
    for x in range(0, len(keys), num_partitions):
        chunk = keys[x:num_partitions+x]
        yield(chunk)

In [98]:
def sharded(keys, n):
    # Get the total number of keys
    num_keys = len(keys)
    # Number of partitions left to make
    num_part_left = n   
    # Set indices for start and end of partition chunk
    chunk_start, chunk_end = 0, math.ceil(num_keys/num_part_left)
    # Number of keys left to partition
    num_keys_left = num_keys - chunk_start
    for x in range(0, n):
        # Create partition chunk
        chunk = keys[chunk_start:chunk_end]
        # Reduce remaining partitions by 1
        num_part_left -= 1
        # Set start index to previous end index
        chunk_start = chunk_end
        # Number of keys left to partition
        num_keys_left = num_keys - chunk_end
        # Calculate end index for next chunk
        if num_part_left > 0: # Note: cannot divide by zero
            chunk_end = chunk_start + math.ceil(num_keys_left/(num_part_left))
        else:
            chunk_end = chunk_start + num_keys_left
        yield(chunk)

In [99]:
def balance_partitions(keys, num_partitions):
    # Sort list of keys
#     list(keys).sort()
    key_list = list(keys)
    key_list.sort()
    # Return list of partitions
    return list(sharded(keys, num_partitions))

In [100]:
def balance_partitions(keys, num_partitions):
    # Sort list of keys
    try:
        sorted_keys = keys.sort_values()
    except:
        sorted_keys = keys
        sorted_keys.sort()
    # Return list of partitions
    return list(sharded(sorted_keys, num_partitions))

In [101]:
balance_partitions(some_keys, 3)

[['A', 'A', 'B', 'B', 'C', 'C', 'C', 'E'],
 ['E', 'H', 'H', 'I', 'I', 'L', 'M'],
 ['O', 'R', 'S', 'T', 'T', 'T', 'Y']]

In [102]:
df.shape

(67663, 43)

In [103]:
# balance_partitions(list(df['airline_country']), 2000)

In [104]:
balance_partitions(df['airline_country'], 1000)

[11836    ALASKA
 11879    ALASKA
 11878    ALASKA
 11877    ALASKA
 11876    ALASKA
           ...  
 11882    ALASKA
 11913    ALASKA
 11849    ALASKA
 11847    ALASKA
 11812    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11811    ALASKA
 11810    ALASKA
 11809    ALASKA
 11808    ALASKA
 11807    ALASKA
           ...  
 11915    ALASKA
 11916    ALASKA
 12013    ALASKA
 12012    ALASKA
 12011    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 12010    ALASKA
 12009    ALASKA
 12008    ALASKA
 12007    ALASKA
 12006    ALASKA
           ...  
 11945    ALASKA
 11944    ALASKA
 11943    ALASKA
 11942    ALASKA
 11941    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11940    ALASKA
 11939    ALASKA
 11938    ALASKA
 11937    ALASKA
 11936    ALASKA
           ...  
 11612    ALASKA
 11611    ALASKA
 11610    ALASKA
 11609    ALASKA
 11608    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11607    ALASKA
 11606    ALASKA
 11605    ALAS

In [105]:
def test_thing(keys, nums):
    sorted = keys.sort_values()
    print(test)

In [106]:
test_thing(df['airline_country'], 2000)

<generator object sharded at 0x7fb1945dacf0>


In [107]:
def sharded(keys, num_partitions):
    for x in range(0, len(keys), num_partitions):
        chunk = keys[x:num_partitions+x]
        yield(chunk)

In [108]:
def balance_partitions(keys, num_partitions):
    try:
        sorted_keys = keys.sort_values()
    except:
        sorted_keys = keys
        sorted_keys.sort()
    num_per_group = math.ceil(len(keys) / num_partitions)
    return list(sharded(sorted_keys, num_per_group))

In [109]:
balance_partitions(df['airline_country'], 1000)

[11836    ALASKA
 11879    ALASKA
 11878    ALASKA
 11877    ALASKA
 11876    ALASKA
           ...  
 11882    ALASKA
 11913    ALASKA
 11849    ALASKA
 11847    ALASKA
 11812    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11811    ALASKA
 11810    ALASKA
 11809    ALASKA
 11808    ALASKA
 11807    ALASKA
           ...  
 11915    ALASKA
 11916    ALASKA
 12013    ALASKA
 12012    ALASKA
 12011    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 12010    ALASKA
 12009    ALASKA
 12008    ALASKA
 12007    ALASKA
 12006    ALASKA
           ...  
 11945    ALASKA
 11944    ALASKA
 11943    ALASKA
 11942    ALASKA
 11941    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11940    ALASKA
 11939    ALASKA
 11938    ALASKA
 11937    ALASKA
 11936    ALASKA
           ...  
 11612    ALASKA
 11611    ALASKA
 11610    ALASKA
 11609    ALASKA
 11608    ALASKA
 Name: airline_country, Length: 68, dtype: object,
 11607    ALASKA
 11606    ALASKA
 11605    ALAS